# Лекция 8. Разреженные матрицы и особенности работы с ними

## План на сегодняшнюю лекцию

- Плотные неструктурированные матрицы и распределённое хранение
- Разреженные матрицы и форматы их представления
- Быстрая реализация умножения разреженной матрицы на вектор 

## Плотные матрицы большой размерности

- Если размер матрицы очень большой, то она не помещается в память
- Возможные способы работы с такими матрицами
    - Если матрица **структурирована**, например блочно Тёплицева с Тёплицевыми блоками (в следующих лекциях), тогда возможно сжатое хранение
    - Для неструктурированных матриц помогает **распределённая память**
    - MPI для обработки распределённо хранимых матриц

### Распределённая память и MPI

- Разбиваем матрицу на блоки и храним их на различных машинах
- Каждая машина имеет своё собственное адресное пространство и не может повредить данные на других машинах
- В этом случае машины передают друг другу данные для агрегирования результата вычислений
- [MPI (Message Passing Interface)](https://en.wikipedia.org/wiki/Message_Passing_Interface) – стандарт в параллельных вычислениях с распределённой памятью

### Пример: умножение матрицы на вектор

- Предположим, вы хотите посчитать произведение $Ax$ и матрица $A$ не помещается в памяти
- В этом случае вы можете разбить матрицу на блоки и поместить их на разные машины
- Возможные стратегии:
    - Одномерное деление на блоки использует только строки
    - Двумерное деление на блоки использует и строки и столбцы

#### Пример одномерного деления на блоки
<img src="./1d_block.jpg">

#### Общее время вычисления произведения матрицы на вектор для одномерного разбиения на блоки

- Каждая машина хранит $n / p $ полных строк и $n / p$ элементов вектора $x$
- Общее число операций $n^2 / p$
- Общее время для отправки и записи данных $t_s \log p + t_w n$, где $t_s$ – единица времени на отправку и $t_w$ – единица времени на запись

#### Пример двумерного деления на блоки

<img src="./2d_block.png" width=400>

#### Общее время вычисления умножения матрицы на вектор с использованием двумерного разбиения на блоки

- Каждая машина хранит блок размера $n / \sqrt{p} $ и $n / \sqrt{p}$ элементов вектора
- Общее число операций $n^2 / p$
- Общее время для отправки и записи данных примерно равно $t_s \log p + t_w (n/\sqrt{p}) \log p$, где $t_s$ – единица времени на отправку и $t_w$ – единица времени на запись

### Пакеты с поддержкой распределённого хранения данных

- [ScaLAPACK](http://www.netlib.org/scalapack/)
- [Trilinos](https://trilinos.org/)

В Python вы можете использовать [mpi4py](https://mpi4py.readthedocs.io/en/stable/) для параллельной реализации ваших алгоритмов.

- PyTorch поддерживает распределённое обучение и хранение данных, см подробности [тут](https://pytorch.org/tutorials/intermediate/dist_tuto.html) 

### Резюме про работу с большими плотными неструктурированными матрицами

- Распределённое хранение матриц
- MPI
- Пакеты, которые используют блочные вычисления
- Различные подходы к блочным вычислениям

## Разреженные матрицы

- Ограничением в решении задач линейной алгебры с плотными матрицами является память, требуемая для хранения плотных матриц, $N^2$ элементов.

- Разреженные матрицы, где большинство элементов нулевые позволяют по крайней мере хранить их в памяти.

- Основные вопросы: можем ли мы решать следующие задачи для разреженных матриц?
    - решение линейных систем
    - вычисление собственных значений и собственных векторов
    - вычисление матричных функций

## Приложения разреженных матриц

Разреженные матрицы возникают в следующих областях:

- математическое моделирование и решение уравнений в частных производных
- обработка графов, например анализ социальных сетей
- рекомендательные системы
- в целом там, где отношения между объектами "разрежены".

### Разреженные матрицы помогают в вычислительной теории графов 

- Графы представляют в виде матриц смежности, которые чаще всего разрежены
- Численное решение задач теории графов сводится к операциям с этими разреженными матрицами
    - Кластеризация графа и выделение сообществ
    - Ранжирование
    - Случайные блуждатели
    - И другие....
- Пример: возможно, самый большой доступный граф гиперссылок содержит 3.5 миллиарда веб-страниц и 128 миллиардов гиперссылок, больше подробностей см. [тут](http://webdatacommons.org/hyperlinkgraph/) 
- Различные графы среднего размера для тестирования ваших алгоритмов доступны в [Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/)

### Florida sparse matrix collection

- Большое количество разреженных матриц из различных приложений вы можете найти в [Florida sparse matrix collection](http://www.cise.ufl.edu/research/sparse/matrices/).

In [ ]:
from IPython.display import IFrame
IFrame('http://yifanhu.net/GALLERY/GRAPHS/search.html', 500, 500)

### Разреженные матрицы и глубокое обучение

- DNN имеют очень много параметров
- Некоторые из них могут быть избыточными
- Как уменьшить число параметров без серьёзной потери в точности?
- [Sparse variational dropout method](https://github.com/ars-ashuha/variational-dropout-sparsifies-dnn) даёт существенно разреженные фильтры в DNN почти без потери точности!

## Построение разреженных матриц

- Мы можем генерировать разреженные матрицы с помощью пакета ```scipy.sparse```

- Можно задать матрицы очень большого размера

Полезные функции при создании разреженных матриц:
- для созданий диагональной матрицы с заданными диагоналями ```spdiags```
- Кронекерово произведение (определение будет далее) разреженных матриц ```kron```
- также арифметические операции для разреженных матриц перегружены

### Кронекерово произведение

Для матриц $A\in\mathbb{R}^{n\times m}$ и $B\in\mathbb{R}^{l\times k}$ Кронекерово произведение определяется как блочная матрица следующего вида

$$
   A\otimes B = \begin{bmatrix}a_{11}B & \dots & a_{1m}B \\ \vdots & \ddots & \vdots \\ a_{n1}B & \dots & a_{nm}B\end{bmatrix}\in\mathbb{R}^{nl\times mk}.
$$

Основные свойства:
- билинейность
- $(A\otimes B) (C\otimes D) = AC \otimes BD$
- Пусть $\mathrm{vec}(X)$ оператор векторизации матрицы по столбцам. Тогда 
$\mathrm{vec}(AXB) = (B^T \otimes A) \mathrm{vec}(X).$

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse
from scipy.sparse import csc_matrix, csr_matrix
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.sparse.linalg
%matplotlib inline
n = 5
ex = np.ones(n);
lp1 = sp.sparse.spdiags(np.vstack((ex,  -2*ex, ex)), [-1, 0, 1], n, n, 'csr'); 
e = sp.sparse.eye(n)
A = sp.sparse.kron(lp1, e) + sp.sparse.kron(e, lp1)
A = csc_matrix(A)
plt.spy(A, aspect='equal', marker='.', markersize=5)

### Шаблон разреженности

- Команда ```spy``` рисует шаблон разреженности данной матрицы: пиксель $(i, j)$ отображается на рисунке, если соответствующий элемент матрицы ненулевой.

- Шаблон разреженности действительно очень важен для понимания сложности алгоритмов линейной алгебры для разреженных матриц.  

- Зачастую шаблона разреженности достаточно для анализа того, насколько "сложно" работать с этой матрицей.

### Определение разреженных матриц

- Разреженные матрицы – это матрицы, такие что количество ненулевых элементов в них существенно меньше общего числа элементов в матрице. 

- Из-за этого вы можете выполнять базовые операции линейной алгебры (прежде всего решать линейные системы) гораздо быстрее по сравнению с использованием плотных матриц.

## Что нам необходимо, чтобы увидеть, как это работает

- **Вопрос 1:** Как хранить разреженные матрицы в памяти?

- **Вопрос 2:** Как умножить разреженную матрицу на вектор быстро?

- **Вопрос 3:** Как быстро решать линейные системы с разреженными матрицами?

### Хранение разреженных матриц

Существет много форматов хранения разреженных матриц, наиболее важные:

- COO (координатный формат)
- LIL (список списков)
- CSR (compressed sparse row)
- CSC (compressed sparse column)
- блочные варианты

В ```scipy``` представлены конструкторы для каждого из этих форматов, например

```scipy.sparse.lil_matrix(A)```.

#### Координатный формат (COO)

- Простейший формат хранения разреженной матрицы – координатный. 
- В этом формате разреженная матрица – это набор индексов и значений в этих индексах.

```python
i, j, val
```

где ```i, j``` массивы индексов, ```val``` массив элементов матрицы. <br>

- Таким образом, нам нужно хранить $3\cdot$**nnz** элементов, где **nnz** обозначает число ненулевых элементов в матрице.

**Q:** Что хорошего и что плохого в использовании такого формата?

#### Основные недостатки

- Он неоптимален по памяти (почему?)
- Он неоптимален для умножения матрицы на вектор (почему?)
- Он неоптимален для удаления элемента (почему?)

Первые два недостатка решены в формате CSR.

**Q**: какой формат решает третий недостаток? 

#### Compressed sparse row (CSR)

В формате CSR матрица хранится также с помощью трёх массивов, но других:

```python
ia, ja, sa
```

где:

- **ia** (начало строк) массив целых чисел длины $n+1$ 
- **ja** (индексы столбцов) массив целых чисел длины **nnz** 
-  **sa** (элементы матрицы) массив действительных чисел длины **nnz**

<img src="https://www.karlrupp.net/wp-content/uploads/2016/02/csr_storage_sparse_marix.png" width=60% />

Итак, всего необходимо хранить $2\cdot{\bf nnz} + n+1$ элементов.

### Разреженные матрицы в PyTorch, Tensorflow и JAX

- PyTorch поддерживает разреженные матрицы в формате COO
- Неполная поддержка вычисления градиентов в операциях с такими матрицами, список и обсуждение см. [тут](https://github.com/pytorch/pytorch/issues/9674)
- Tensorflow также поддерживает разреженные матрицы в COO формате
- Список поддерживаемых операций приведён [здесь](https://www.tensorflow.org/api_docs/python/tf/sparse), поддержка вычисления градиентов ограничена
- JAX также имеет [экспериментальную поддержку](https://jax.readthedocs.io/en/latest/jax.experimental.sparse.html) операций с разреженными матрицами

### CSR формат позволяет быстро умножить разреженную матрицу на вектор (SpMV)

```python

   for i in range(n):
        
        for k in range(ia[i]:ia[i+1]):
            
            y[i] += sa[k] * x[ja[k]]
```

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse import csc_matrix, csr_matrix, coo_matrix
import matplotlib.pyplot as plt
%matplotlib inline
n = 1000
ex = np.ones(n);
lp1 = sp.sparse.spdiags(np.vstack((ex,  -2*ex, ex)), [-1, 0, 1], n, n, 'csr'); 
e = sp.sparse.eye(n)
A = sp.sparse.kron(lp1, e) + sp.sparse.kron(e, lp1)
A = csr_matrix(A)
rhs = np.ones(n * n)
B = coo_matrix(A)
%timeit A.dot(rhs)
%timeit B.dot(rhs)

Видно, что **CSR** быстрее, и чем менее структурирован шаблон разреженности, тем выше выигрыш в скорости.

### Разреженные матрицы и эффективность

- Использование разреженных матриц приводит к уменьшению сложности
- Но они не очень подходят для параллельных/GPU реализаций 
- Они не показывают максимальную эффективность из-за случайного доступа к данным.  
- Обычно, пиковая производительность порядка $10\%-15\%$ считается хорошей.  

### Вспомним как измеряется эффективность операций

- Стандартный способ измерения эффективности операций линейной алгебры – это использование **flops** (число опраций с плавающей точкой в секунду)

- Измерим эффективность умножения матрицы на вектор в случае плотной и разреженной матрицы

In [ ]:
import numpy as np
import time
n = 4000
a = np.random.randn(n, n)
v = np.random.randn(n)
t = time.time()
np.dot(a, v)
t = time.time() - t
print('Time: {0: 3.1e}, Efficiency: {1: 3.1e} Gflops'.\
      format(t,  ((2 * n ** 2)/t) / 10 ** 9))

In [ ]:
n = 4000
ex = np.ones(n);
a = sp.sparse.spdiags(np.vstack((ex,  -2*ex, ex)), [-1, 0, 1], n, n, 'csr'); 
rhs = np.random.randn(n)
t = time.time()
a.dot(rhs)
t = time.time() - t
print('Time: {0: 3.1e}, Efficiency: {1: 3.1e} Gflops'.\
      format(t,  (3 * n) / t / 10 ** 9))

### Случайный доступ к данным и промахи в обращении к кешу

- Сначала все элементы матрицы и вектора хранятся в оперативной памяти (RAM – Random Access Memory)
- Если вы хотите вычислить произведение матрицы на вектор, часть элементов матрицы и вектора перемещаются в кеш (быстрой памяти малого объёма), см. [лекцию об алгоритме Штрассена и умножении матриц](https://github.com/amkatrutsa/nla2020_ozon/blob/master/lectures/lecture4/lecture4.ipynb)
- После этого CPU берёт данные из кеша, обрабатывает их и возвращает результат снова в кеш
- Если CPU требуются данные, которых ещё нет в кеше, это называется промах в обращении к кешу (cache miss)
- Если случается промах в обращении к кешу, необходимые данные перемещаются из оперативной памяти в кеш

**Q**: что если в кеше нет свободного места?


- Чем больше промахов в обращении к кешу, тем медленнее выполняются вычисления

### План кеша и LRU

<img src="./cache_scheme.png" width="500">

#### Умножение матрицы в CSR формате на вектор

```python

   for i in range(n):
        
        for k in range(ia[i]:ia[i+1]):
            
            y[i] += sa[k] * x[ja[k]]
            
```

- Какая часть операций приводит к промахам в обращении к кешу?
- Как эту проблему можно решить?

### Переупорядочивание уменьшает количество промахов в обращении к кешу

- Если ```ja``` хранит последовательно элементы, тогда они могут быть перемещены в кеш одновременно и количество промахов в обращении к кешу уменьшится
- Так происходит, когда разреженная матрица является **ленточной** или хотя бы блочно-диагональной
- Мы можем превратить данную разреженную матрицу в ленточную или блочно-диагональную с помощью *перестановок* 

- Пусть $P$ матрица перестановок строк матрицы и $Q$ матрица перестановок столбцов матрицы
- $A_1 = PAQ$ – матрица с шириной ленты меньшей, чем у матрицы $A$
- $y = Ax \to \tilde{y} = A_1 \tilde{x}$, где $\tilde{x} = Q^{\top}x$ и $\tilde{y} = Py$
- [Separated block diagonal form](http://albert-jan.yzelman.net/PDFs/yzelman09-rev.pdf) призван минимизировать количество промахов в обращении к кешу
- Он также может быть расширен на двумерный случай, где разделяются не только строки, но и столбцы

#### Пример

- SBD в одномерном случае
<img src="./sbd.png" width="400">

## Методы решения линейных систем с разреженными матрицами

- Прямые методы
    - LU разложение
    - Различные методы переупорядочивания для минимизации заполнения факторов
- Крыловские методы

In [ ]:
n = 10
ex = np.ones(n);
lp1 = sp.sparse.spdiags(np.vstack((ex,  -2*ex, ex)), [-1, 0, 1], n, n, 'csr'); 
e = sp.sparse.eye(n)
A = sp.sparse.kron(lp1, e) + sp.sparse.kron(e, lp1)
A = csr_matrix(A)
rhs = np.ones(n * n)
sol = sp.sparse.linalg.spsolve(A, rhs)
_, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(sol)
ax1.set_title('Not reshaped solution')
ax2.contourf(sol.reshape((n, n), order='f'))
ax2.set_title('Reshaped solution')

## LU разложение разреженной матрицы

- Почему разреженная линейная система может быть решена быстрее, чем плотная? С помощью какого метода? 

- В LU разложении матрицы $A$ факторы $L$ и $U$ могут быть также разреженными:

$$A = L U$$

- А решение линейной системы с разреженной треугольной матрицей может быть вычислено очень быстро.  

<font color='red'> Заметим, что обратная матрица от разреженной матрицы НЕ разрежена! </font>


In [ ]:
n = 7
ex = np.ones(n);
a = sp.sparse.spdiags(np.vstack((ex,  -2*ex, ex)), [-1, 0, 1], n, n, 'csr'); 
b = np.array(np.linalg.inv(a.toarray()))
print(a.toarray())
print(b)

## А факторы...

- $L$ и $U$ обычно разрежены
- В случае трёхдиагональной матрицы они даже бидиагональны!

In [ ]:
from scipy.sparse.linalg import splu
T = splu(a.tocsc(), permc_spec="NATURAL")
plt.spy(T.L)

Отметим, что ```splu``` со значением параметра ```permc_spec``` по умолчанию даёт перестановку, которая не даёт бидиагональные факторы:

In [ ]:
from scipy.sparse.linalg import splu
T = splu(a.tocsc())
plt.spy(T.L)
print(T.perm_c)

## Двумерный случай

В двумерном случае всё гораздо хуже:

In [ ]:
n = 20
ex = np.ones(n);
lp1 = sp.sparse.spdiags(np.vstack((ex,  -2*ex, ex)), [-1, 0, 1], n, n, 'csr'); 
e = sp.sparse.eye(n)
A = sp.sparse.kron(lp1, e) + sp.sparse.kron(e, lp1)
A = csc_matrix(A)
T = scipy.sparse.linalg.spilu(A)
plt.spy(T.L, marker='.', color='k', markersize=8)

## План на следующую лекцию

- Как добиться максимально разреженных факторов?
- Почему методы из теории графов тут помогают?
- Что такое лапласиан графа, вектор Фидлера и почему они важны?

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()